In [17]:
# Import libraries
import os
from tqdm import tqdm
import pandas as pd

# Redirecting to the directory
# os.chdir(os.path.join('.\ECON4999X'))
CLEANED_DATA_DIR = os.path.normpath(r'.\Cleaned_Data')
DA_DATA_DIR = os.path.normpath(r'.\da_working_files')
print(os.listdir(CLEANED_DATA_DIR))

['df_exchange_info_master.xlsx', 'df_exchange_mapping.csv', 'df_ghg_ts_long_form.csv', 'df_ghg_ts_long_form_selected.csv', 'df_ghg_ts_master.csv', 'df_info_master.csv', 'df_power_ts_long_form.csv', 'df_power_ts_long_form_selected.csv', 'df_power_ts_master.csv', 'df_selected_comps.csv', 'hardcode_data.xlsx', '~$hardcode_data.xlsx']


### Does higher carbon price reduce firms’ carbon emissions?
#### Emission Analysis

In [14]:
# Reading dataframe
df_ghg = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_ghg_ts_long_form_selected.csv'))
df_ghg.head()

# We want to make a dataframe that only contains entity with long enough history
entity_history_requirement = 15
df_ghg_long_hist_comps = df_ghg.groupby('entity_id').filter(lambda x: len(x['year']) >= entity_history_requirement)

In [20]:
# Create a column called the total carbon emission
df_ghg_long_hist_comps['total_carbon_emission'] = df_ghg_long_hist_comps[['absolute_ghg_scope_1', 'absolute_ghg_scope_2']].sum(axis=1)

In [24]:
file_name = 'df_ghg_ts_long_form_selected_long_hist.csv'
df_ghg_long_hist_comps.to_csv(os.path.join(DA_DATA_DIR, file_name), index=False)

### How firms reduce the carbon emissions, by developing in-house green technology or cutting their output?
#### Only those companies shows reduction in carbon emission

In [48]:
# Getting only the firms which show reduction in carbon emission
df_ghg = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_ghg_ts_long_form_selected.csv'))
df_ghg.head()

df_ghg_reduce_carbon_emission = df_ghg.groupby('entity_id').filter(lambda x: x.iloc[0]['total_carbon_emission'] > x.iloc[-1]['total_carbon_emission'])

In [49]:
# Merging for their respective power consumption category
df_power = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_power_ts_long_form_selected.csv'))
df_power.head()

,entity_id,year,biomass,coal,geothermal,hydroelectric,landfill_gas,liquid_nat_gas,nat_gas,nuclear,petroleum_oil,solar,wave_and_tidal,wind
0,5001248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4147764,2012.0,450.0,NaN,NaN,1828.0,NaN,NaN,28.0,NaN,NaN,697.0,NaN,17376.0
2,4147764,2013.0,454.0,NaN,NaN,2882.0,NaN,NaN,28.0,NaN,NaN,641.0,NaN,18399.0
3,4147764,2014.0,405.0,NaN,NaN,2933.0,NaN,NaN,1.0,NaN,NaN,629.0,NaN,17482.0
4,4147764,2015.0,408.0,NaN,NaN,2197.0,NaN,NaN,NaN,NaN,NaN,849.0,NaN,17421.0


In [50]:
df_ghg_reduce_carbon_emission = df_ghg_reduce_carbon_emission.merge(df_power, on=['entity_id', 'year'], how='left')

In [51]:
# Define the green tech power generation
green_tech = ['biomass', 'geothermal', 'hydroelectric', 'landfill_gas',
            'nuclear','solar', 'wave_and_tidal', 'wind']
fossil_fuel = ['nat_gas', 'petroleum_oil', 'liquid_nat_gas', 'coal']

In [52]:
df_ghg_reduce_carbon_emission['green_tech_power'] = df_ghg_reduce_carbon_emission[green_tech].sum(axis=1)
df_ghg_reduce_carbon_emission['fossil_fuel_power'] = df_ghg_reduce_carbon_emission[fossil_fuel].sum(axis=1)
df_ghg_reduce_carbon_emission['total_power'] = df_ghg_reduce_carbon_emission['fossil_fuel_power']+df_ghg_reduce_carbon_emission['green_tech_power']
df_ghg_reduce_carbon_emission['green_tech_rr'] = df_ghg_reduce_carbon_emission['green_tech_power'] / df_ghg_reduce_carbon_emission['total_power']
df_ghg_reduce_carbon_emission

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,nat_gas,nuclear,petroleum_oil,solar,wave_and_tidal,wind,green_tech_power,fossil_fuel_power,total_power,green_tech_rr
0,4147764,2004.0,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
1,4147764,2005.0,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
2,4147764,2006.0,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
3,4147764,2007.0,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
4,4147764,2008.0,1.175414e+06,167569.980000,66.636061,9.499804,1.342984e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,4539460,2016.0,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,78.00,9192.00,9270.0,0.008414
176,4539460,2017.0,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.67,8353.33,8390.0,0.004371
177,4539460,2018.0,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.22,6009.78,6050.0,0.006648
178,4539460,2019.0,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,63.82,3336.18,3400.0,0.018771


In [57]:
df_ghg_reduce_carbon_emission = df_ghg_reduce_carbon_emission.replace(0, float('nan'))

In [58]:
file_name = 'df_ghg_ts_long_form_reduced_carbon_emission.csv'
df_ghg_reduce_carbon_emission.to_csv(os.path.join(DA_DATA_DIR, file_name), index=False)

#### Loosen the assumption
- Including all the companies in Europe, with dummy variable of is_reduced_ce

In [69]:
df_ghg = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_ghg_ts_long_form_selected.csv'))
# Merging for their respective power consumption category
df_power = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_power_ts_long_form_selected.csv'))

In [70]:
df_panel_master = df_ghg.merge(df_power, on=['entity_id', 'year'], how='left')
df_panel_master

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,hydroelectric,landfill_gas,liquid_nat_gas,nat_gas,nuclear,petroleum_oil,solar,wave_and_tidal,wind
0,5001248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4147764,2004.0,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4147764,2005.0,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4147764,2006.0,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4147764,2007.0,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,4539460,2016.0,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,4539460,2017.0,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,4539460,2018.0,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,4539460,2019.0,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df_panel_master['green_tech_power'] = df_panel_master[green_tech].sum(axis=1)
df_panel_master['fossil_fuel_power'] = df_panel_master[fossil_fuel].sum(axis=1)
df_panel_master['total_power'] = df_panel_master['fossil_fuel_power']+df_panel_master['green_tech_power']
df_panel_master['green_tech_rr'] = df_panel_master['green_tech_power'] / df_panel_master['total_power']
df_panel_master

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,nat_gas,nuclear,petroleum_oil,solar,wave_and_tidal,wind,green_tech_power,fossil_fuel_power,total_power,green_tech_rr
0,5001248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
1,4147764,2004.0,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
2,4147764,2005.0,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
3,4147764,2006.0,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
4,4147764,2007.0,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,4539460,2016.0,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,78.00,9192.00,9270.0,0.008414
290,4539460,2017.0,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.67,8353.33,8390.0,0.004371
291,4539460,2018.0,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.22,6009.78,6050.0,0.006648
292,4539460,2019.0,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,63.82,3336.18,3400.0,0.018771


In [77]:
tickers_reduced_ce = df_ghg_reduce_carbon_emission.entity_id.unique()
tickers_long_hist = df_ghg_long_hist_comps.entity_id.unique()

In [78]:
df_panel_master['is_reduced_ce'] = df_panel_master['entity_id'].isin(tickers_reduced_ce)
df_panel_master['is_long_hist'] = df_panel_master['entity_id'].isin(tickers_long_hist)

In [79]:
df_panel_master

,entity_id,year,absolute_ghg_scope_1,absolute_ghg_scope_2,intensity_ghg_scope_1,intensity_ghg_scope_2,total_carbon_emission,biomass,coal,geothermal,...,petroleum_oil,solar,wave_and_tidal,wind,green_tech_power,fossil_fuel_power,total_power,green_tech_rr,is_reduced_ce,is_long_hist
0,5001248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN,False,False
1,4147764,2004.0,1.360592e+06,105613.412500,247.430227,19.206315,1.466205e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN,True,True
2,4147764,2005.0,1.289793e+06,100051.889800,225.311038,17.477839,1.389845e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN,True,True
3,4147764,2006.0,1.670383e+06,126135.647900,201.841385,15.241652,1.796518e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN,True,True
4,4147764,2007.0,1.586784e+06,35446.030000,136.679404,3.053184,1.622230e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,0.00,0.0,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,4539460,2016.0,1.161410e+07,903.453175,16951.953960,1.318682,1.161500e+07,78.00,9192.00,NaN,...,NaN,NaN,NaN,NaN,78.00,9192.00,9270.0,0.008414,True,False
290,4539460,2017.0,1.099803e+07,1160.731868,17021.534380,1.796453,1.099919e+07,36.67,8353.33,NaN,...,NaN,NaN,NaN,NaN,36.67,8353.33,8390.0,0.004371,True,False
291,4539460,2018.0,7.332719e+06,1057.278150,11495.301030,1.657466,7.333776e+06,40.22,6009.78,NaN,...,NaN,NaN,NaN,NaN,40.22,6009.78,6050.0,0.006648,True,False
292,4539460,2019.0,4.226335e+06,968.032085,6098.313233,1.396804,4.227303e+06,63.82,3336.18,NaN,...,NaN,NaN,NaN,NaN,63.82,3336.18,3400.0,0.018771,True,False


In [81]:
df_panel_master = df_panel_master.replace(0, float('nan'))
file_name = 'df_panel_master.csv'
df_panel_master.to_csv(os.path.join(DA_DATA_DIR, file_name), index=False)

AttributeError: 'DataFrame' object has no attribute 'repalce'